# 國發會 - 景氣指標及燈號  
* [國發會 - 景氣指標查詢系統](https://index.ndc.gov.tw/n/zh_tw)  
* [政府資料開放平臺 - ZIP格式](https://data.gov.tw/dataset/6099)  
* [台灣-景氣對策信號vs.台股_財經Ｍ平方](https://www.macromicro.me/collections/10/tw-monitoring-indicators-relative/90/tw-light-stock)  

In [1]:
import os
import sys
from pathlib import Path
from datetime import date
from datetime import datetime
import shutil
import requests
import pandas as pd
import duckdb
import zipfile

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

## 公用參數設定

In [ ]:
DOWNLOAD_FOLDER = os.environ.get('download_path')
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

f"下載資料存放路徑: {DOWNLOAD_FOLDER}"

## 外部資料讀取

In [4]:
# 透過網址下載資料來源zip檔
data_url = "https://ws.ndc.gov.tw/Download.ashx?u=LzAwMS9hZG1pbmlzdHJhdG9yLzEwL3JlbGZpbGUvNTc4MS82MzkyL2VhMjM1YmQ5LWQwNTItNGE2OS1hYmZjLWQ1Yzc4NWQzZDBlMi56aXA%3d&n=5pmv5rCj5oyH5qiZ5Y%2bK54eI6JmfLnppcA%3d%3d&icon=.zip"

# 設定下載檔案存放路徑
data_path = os.path.join(DOWNLOAD_FOLDER, "tw_stock.zip")

# 透過Requests下載資料來源zip檔
response = requests.get(data_url)
with open(data_path, 'wb') as f:
    f.write(response.content)

# 解壓縮下載的zip檔
with zipfile.ZipFile(data_path, 'r') as zip_ref:
    zip_ref.extractall(DOWNLOAD_FOLDER)

In [ ]:
# 讀取景氣燈號資料
rep_path = os.path.join(DOWNLOAD_FOLDER, "景氣指標與燈號.csv")
indicator_df = pd.read_csv(rep_path, encoding='utf-8', dtype=str)
indicator_df

In [ ]:
# 讀取景氣信號構成項目資料
rep_path = os.path.join(DOWNLOAD_FOLDER, "景氣對策信號構成項目.csv")
indicator_items_df = pd.read_csv(rep_path, encoding='utf-8', dtype=str)
indicator_items_df

## 資料留存ＤＢ

In [7]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [8]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

### 景氣燈號資料留存

In [9]:
table_name = "tw_business_indicators"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    indicator_df, 
    table_name, 
    date_column='Date',
    if_exists='replace'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 確認寫入結果
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

### 景氣信號構成項目資料

In [12]:
table_name = "tw_business_indicators_items"

In [ ]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    indicator_items_df, 
    table_name, 
    date_column='Date',
    if_exists='replace'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

In [ ]:
# 確認寫入結果
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

In [15]:
# 關閉資料庫連線
conn_duckdb.close()

## 刪除下載暫存檔案

In [16]:
def delete_all_files_in_directory(directory_path):
    # 確認目錄是否存在
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        # 列出目錄中的所有檔案
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                # 檢查是否是檔案然後刪除
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                # 如果是目錄，可以用shutil.rmtree刪除
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Failed to delete {file_path}. Reason: {e}')
        print(f"All files in directory '{directory_path}' have been deleted.")
    else:
        print(f"The directory '{directory_path}' does not exist or is not a directory.")


In [ ]:
# 刪除temp目錄下所有檔案
delete_all_files_in_directory(DOWNLOAD_FOLDER)